In [23]:
# import the necessary packages
import numpy as np
import cv2
import os
from tqdm import tqdm

from keras.preprocessing.image import img_to_array
from keras.models import Sequential
from keras.layers.convolutional import Conv2D
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras import backend as K
from keras.optimizers import SGD


from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from imutils import paths
import matplotlib.pyplot as plt
import numpy as np
import argparse


In [2]:
# import the necessary packages

class SimplePreprocessor:
    def __init__(self, width, height, inter=cv2.INTER_AREA):
        # store the target image width, height, and interpolation
        # method used when resizing
        self.width = width
        self.height = height
        self.inter = inter

    def preprocess(self, image):
        # resize the image to a fixed size, ignoring the aspect
        # ratio
        return cv2.resize(image, (self.width, self.height),interpolation=self.inter)

In [3]:
# import the necessary packages

class ImageToArrayPreprocessor:
    def __init__(self, dataFormat=None):
        # store the image data format
        self.dataFormat = dataFormat

    def preprocess(self, image):
        # apply the Keras utility function that correctly rearranges
        # the dimensions of the image
#         return img_to_array(image, data_format=self.dataFormat)
        return img_to_array(image)

In [7]:


class SimpleDatasetLoader:
#     def __init__(self, preprocessors=None):
    def __init__(self, width, height):
        from tqdm import tqdm
        self.width = width
        self.height = height



    def load(self, imagePaths, verbose=-1):
        # initialize the list of features and labels
        data = []
        labels = []

        for img in tqdm(os.listdir(imagePaths)):
            label = img
            path = os.path.join(imagePaths,img)
            for filename in os.listdir(path):
                image = cv2.imread(os.path.join(path, filename))
                sp = SimplePreprocessor(self.width, self.height)
                image = sp.preprocess(image)
#                 print ("sp completed")
                iap = ImageToArrayPreprocessor()
                image = iap.preprocess(image)

                data.append(image)
                labels.append(label)

        return (np.array(data), np.array(labels))


In [40]:
class  Shallow_Net:
    
    def build(width, height, depth, classes):
        
        model = Sequential()
        inputShape = (width, height, depth)
        
        if K.image_data_format =="channel_first":
            inputShape = (depth, width, height)
        
        model.add(Conv2D(filters = 32, kernel_size = (3,3), padding="same", input_shape = inputShape))
        model.add(Activation("softmax"))
        
        model.add(Flatten())
        model.add(Dense(classes))
        model.add(Activation("softmax"))
        
                  
        return model

In [1]:
print("[INFO] loading images...")
imagePaths = 'G:/pyimagesearch/SB_Code/SB_Code/datasets/animals/'

sdl = SimpleDatasetLoader(width = 32, height = 32)
(data, labels) = sdl.load(imagePaths, verbose=500)
data = data.astype("float") / 255.0
print ('completed')

In [35]:
type(labels)

numpy.ndarray

In [36]:
np.unique(labels, return_counts=True)


(array(['cats', 'dogs', 'panda'], dtype='<U5'),
 array([1000, 1000, 1000], dtype=int64))

In [37]:
(trainX, testX, trainY, testY) = train_test_split(data, labels,
	test_size=0.25, random_state=42)



In [38]:
# convert the labels from integers to vectors
trainY = LabelBinarizer().fit_transform(trainY)
testY = LabelBinarizer().fit_transform(testY)


In [43]:
# initialize the optimizer and model
print("[INFO] compiling model...")
opt = SGD(lr=0.005)
model = Shallow_Net.build(width=32, height=32, depth=3, classes=3)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

[INFO] compiling model...


In [45]:
# train the network
print("[INFO] training network...")
H = model.fit(trainX, trainY, validation_data=(testX, testY), batch_size=64, epochs = 40, verbose=2)

[INFO] training network...
Train on 2250 samples, validate on 750 samples
Epoch 1/40
 - 8s - loss: 1.0965 - acc: 0.3533 - val_loss: 1.0973 - val_acc: 0.3320
Epoch 2/40
 - 8s - loss: 1.0959 - acc: 0.3778 - val_loss: 1.0959 - val_acc: 0.3320
Epoch 3/40
 - 7s - loss: 1.0950 - acc: 0.3916 - val_loss: 1.0957 - val_acc: 0.3200
Epoch 4/40
 - 8s - loss: 1.0941 - acc: 0.4191 - val_loss: 1.0935 - val_acc: 0.3893
Epoch 5/40
 - 8s - loss: 1.0929 - acc: 0.3880 - val_loss: 1.0925 - val_acc: 0.3320
Epoch 6/40
 - 7s - loss: 1.0924 - acc: 0.3773 - val_loss: 1.0921 - val_acc: 0.4147
Epoch 7/40
 - 7s - loss: 1.0914 - acc: 0.4004 - val_loss: 1.0905 - val_acc: 0.4747
Epoch 8/40
 - 7s - loss: 1.0900 - acc: 0.4631 - val_loss: 1.0909 - val_acc: 0.3533
Epoch 9/40
 - 8s - loss: 1.0891 - acc: 0.4587 - val_loss: 1.0881 - val_acc: 0.4067
Epoch 10/40
 - 8s - loss: 1.0881 - acc: 0.4804 - val_loss: 1.0874 - val_acc: 0.4320
Epoch 11/40
 - 9s - loss: 1.0871 - acc: 0.4782 - val_loss: 1.0868 - val_acc: 0.4867
Epoch 12/40

In [46]:
# evaluate the network
print("[INFO] evaluating network...")
predictions = model.predict(testX, batch_size=64)
print(classification_report(testY.argmax(axis=1), predictions.argmax(axis=1), target_names=["cat", "dog", "panda"]))


[INFO] evaluating network...
             precision    recall  f1-score   support

        cat       0.53      0.32      0.40       262
        dog       0.45      0.51      0.48       249
      panda       0.61      0.79      0.69       239

avg / total       0.53      0.53      0.52       750

